In [1]:
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import pandas as pd

# Load the LiveLab.csv file
livelab = pd.read_csv('PreHelp_LiveLab.csv')

# Load analysis.txt with explicit column names (adjust delimiter if needed)
analysis = pd.read_csv("Analysis-gaming-clip4-x019160_output.txt", delim_whitespace=True, header=None, names=["Student_ID", "Time", "Detector_Name", "Gaming", "Actual_Value"])

print(analysis.head(10))

# Ensure we use the real gaming value
analysis["Gaming"] = analysis["Actual_Value"]

# Drop unnecessary columns
analysis = analysis.drop(columns=["Detector_Name", "Actual_Value"])

# Rename columns for consistency with livelab.csv
analysis = analysis.rename(columns={"Student_ID": "Anon Student Id", "Time": "Time"})

# Standardize time format
livelab["Time"] = pd.to_datetime(livelab["Time"])
analysis["Time"] = pd.to_datetime(analysis["Time"])

# Create a mapping of (Student ID, Time) → Gaming status
gaming_dict = {(row["Anon Student Id"], row["Time"]): row["Gaming"] for _, row in analysis.iterrows()}

# Map the correct gaming status to livelab.csv
livelab["Gaming"] = livelab.apply(lambda row: gaming_dict.get((row["Anon Student Id"], row["Time"]), "Not gaming"), axis=1)

# Convert "Gaming" column to boolean (True if "gaming", False otherwise)
livelab["Gaming"] = livelab["Gaming"].apply(lambda x: True if x.lower() == "gaming" else False)

# Save the updated file
# livelab.to_csv("LiveLab_updated.csv", index=False)

print(livelab["Gaming"])  # Prints each value on a new line

# print("Updated LiveLab.csv saved as LiveLab_updated.csv")


/var/folders/bh/d4vzbc194yxb1c729qdk9r940000gn/T/ipykernel_34198/2487516746.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  analysis = pd.read_csv("Analysis-gaming-clip4-x019160_output.txt", delim_whitespace=True, header=None, names=["Student_ID", "Time", "Detector_Name", "Gaming", "Actual_Value"])


   Student_ID                      Time  Detector_Name Gaming Actual_Value
0  Student_ID                      Time  Detector_Name  Value          NaN
1        s141  2023-03-24T14:27:09.000Z         gaming    Not       gaming
2        s293  2023-02-08T18:19:46.000Z         gaming    Not       gaming
3        s316  2023-03-15T16:10:20.000Z         gaming    Not       gaming
4        s295  2023-03-13T09:24:37.000Z         gaming    Not       gaming
5        s117  2023-03-02T17:17:26.000Z         gaming    Not       gaming
6          s5  2023-02-27T15:34:01.000Z         gaming    Not       gaming
7        s282  2023-03-01T15:41:04.000Z         gaming    Not       gaming
8        s280  2023-03-07T17:18:02.000Z         gaming    Not       gaming
9        s207  2023-02-27T20:49:14.000Z         gaming    Not       gaming


/var/folders/bh/d4vzbc194yxb1c729qdk9r940000gn/T/ipykernel_34198/2487516746.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  analysis["Time"] = pd.to_datetime(analysis["Time"])


DateParseError: Unknown datetime string format, unable to parse: Time, at position 0

In [4]:
import pandas as pd

# Read the LiveLab.csv file
livelab_df = pd.read_csv('PreHelp_LiveLab.csv')

# Read the analysis.txt file
# Assuming the file is space-separated and has columns: Student_ID, Time, Detector_Name, Value, True_Value
analysis_df = pd.read_csv('Analysis-gaming-clip4-x019160_output.txt', sep='\s+', header=None, names=['Student_ID', 'Time', 'Detector_Name', 'Value', 'True_Value'])

# Create a dictionary to map (Student_ID, Time) to gaming status
gaming_status = {}
for index, row in analysis_df.iterrows():
    student_id = row['Student_ID']
    time = row['Time']
    # The True_Value column contains the actual status (e.g., "Not gaming")
    status = row['True_Value'].strip() == 'Gaming'  # True if 'gaming', False otherwise
    gaming_status[(student_id, time)] = status

# Add a new column to LiveLab.csv to store the gaming status
livelab_df['Gaming'] = livelab_df.apply(lambda row: gaming_status.get((row['Anon Student Id'], row['Time']), False), axis=1)

# Save the updated DataFrame to a new CSV file
# livelab_df.to_csv('LiveLab_updated.csv', index=False)

print(livelab_df['Gaming'])  # Prints each value on a new line


AttributeError: 'float' object has no attribute 'strip'

In [13]:
import pandas as pd

# Load LiveLab.csv
livelab_df = pd.read_csv('PreHelp_LiveLab.csv')

# Load analysis.txt, ensuring correct spacing
analysis_df = pd.read_csv('Analysis-gaming-clip4-x019160_output.txt', delim_whitespace=True, header=0)

# Manually correct misalignment if necessary
if "Value" in analysis_df.columns:
    analysis_df = analysis_df.drop(columns=["Value"])  # Drop the redundant column

# Rename the last column to "Gaming" if it's misaligned
column_names = list(analysis_df.columns)
if column_names[-1] != "Gaming":  # Ensure the last column is named correctly
    analysis_df.rename(columns={column_names[-1]: "Gaming"}, inplace=True)

# Ensure "Gaming" column values are correctly mapped to boolean
analysis_df["Gaming"] = analysis_df["Gaming"].str.strip() != "Not gaming"

# Strip any whitespace from column names
livelab_df.columns = livelab_df.columns.str.strip()
analysis_df.columns = analysis_df.columns.str.strip()

# Strip whitespace from student IDs (if any)
livelab_df["Anon Student Id"] = livelab_df["Anon Student Id"].astype(str).str.strip()
analysis_df["Student_ID"] = analysis_df["Student_ID"].astype(str).str.strip()

analysis_df.head()

# Convert "Time" columns to datetime format
livelab_df["Time"] = pd.to_datetime(livelab_df["Time"])
analysis_df["Time"] = pd.to_datetime(analysis_df["Time"])

# Merge LiveLab with analysis.txt based on Student ID and Time
merged_df = livelab_df.merge(analysis_df, left_on=["Anon Student Id", "Time"], right_on=["Student_ID", "Time"], how="left")

# Drop redundant Student_ID column from analysis.txt after merging
merged_df.drop(columns=["Student_ID"], inplace=True)

# Fill missing values in "Gaming" column with False (if no match was found)
merged_df["Gaming"].fillna(False, inplace=True)

# Save the updated DataFrame to a new CSV file
# merged_df.to_csv("Updated_LiveLab.csv", index=False)

# Display a sample of the final dataframe
# merged_df.head()

# Print values where Gaming is True
gaming_true_df = merged_df[merged_df["Gaming"] == True]
print(gaming_true_df)

/var/folders/bh/d4vzbc194yxb1c729qdk9r940000gn/T/ipykernel_34198/3054721702.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  analysis_df = pd.read_csv('Analysis-gaming-clip4-x019160_output.txt', delim_whitespace=True, header=0)


DateParseError: Unknown datetime string format, unable to parse: gaming, at position 0

In [6]:
import pandas as pd

# Read the analysis.txt file as plain text
with open('Analysis-gaming-clip4-x019160_output.txt', 'r') as file:
    lines = file.readlines()

# Parse the lines to extract Student_ID, Time, Detector_Name, and Gaming_Status
parsed_data = []
for line in lines:
    # Split the line by tabs (assuming it's tab-separated)
    parts = line.strip().split('\t')
    
    # Ensure there are at least 4 parts (Student_ID, Time, Detector_Name, Value)
    if len(parts) >= 4:
        student_id = parts[0]
        time = parts[1]
        detector_name = parts[2]
        value = parts[3]  # This will always be "gaming"
        
        # Extract the true gaming status (the text after "gaming")
        gaming_status = line.strip().split('gaming')[-1].strip()
        
        # Append the parsed data to the list
        parsed_data.append([student_id, time, detector_name, gaming_status])

# Create a DataFrame from the parsed data
analysis_df = pd.DataFrame(parsed_data, columns=['Student_ID', 'Time', 'Detector_Name', 'Gaming_Status'])

# Debug: Print parsed analysis data
print("Parsed Analysis Data:")
print(analysis_df)

# Create a dictionary to map (Student_ID, Time) to the gaming status
gaming_dict = {(row['Student_ID'], row['Time']): row['Gaming_Status'].strip() == 'gaming' 
               for _, row in analysis_df.iterrows()}

# Debug: Print gaming dictionary
print("Gaming Dictionary:")
for key, value in gaming_dict.items():
    print(key, "->", value)

# Load the LiveLab.csv file
live_lab_df = pd.read_csv('PreHelp_LiveLab.csv')

# Debug: Print LiveLab data
print("LiveLab Data:")
print(live_lab_df[['Anon Student Id', 'Time']].head())

# Add a new column 'Gaming' to LiveLab.csv based on the gaming_dict
live_lab_df['Gaming'] = live_lab_df.apply(
    lambda row: gaming_dict.get((row['Anon Student Id'], row['Time']), False), axis=1)

# Debug: Print rows where Gaming is True
print("Rows where Gaming is True:")
print(live_lab_df[live_lab_df['Gaming'] == True])

# Save the updated DataFrame back to LiveLab.csv
# live_lab_df.to_csv('LiveLab_updated.csv', index=False)

print("Updated LiveLab.csv with Gaming column saved as LiveLab_updated.csv")

Parsed Analysis Data:
        Student_ID                      Time  Detector_Name  \
0       Student_ID                      Time  Detector_Name   
1             s141  2023-03-24T14:27:09.000Z         gaming   
2             s293  2023-02-08T18:19:46.000Z         gaming   
3             s316  2023-03-15T16:10:20.000Z         gaming   
4             s295  2023-03-13T09:24:37.000Z         gaming   
...            ...                       ...            ...   
100858        s357  2023-04-25T14:59:56.000Z         gaming   
100859        s357  2023-04-25T15:11:47.000Z         gaming   
100860        s357  2023-04-25T15:08:23.000Z         gaming   
100861        s357  2023-04-25T15:05:00.000Z         gaming   
100862        s357  2023-04-25T15:00:56.000Z         gaming   

                                 Gaming_Status  
0       Student_ID\tTime\tDetector_Name\tValue  
1                                               
2                                               
3                        

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



LiveLab Data:
  Anon Student Id                 Time
0            s141  2023-03-24 14:26:50
1            s141  2023-03-24 14:26:53
2            s141  2023-03-24 14:27:01
3            s141  2023-03-24 14:27:07
4            s141  2023-03-24 14:27:09
Rows where Gaming is True:
Empty DataFrame
Columns: [Anon Student Id, Session Id, Time, KC_Model(MATHia), Level (Section), Problem Name, Step Name, Attempt At Step, Help Level, Selection, Action, Input, Outcome, CF (Rule Id), CF (Semantic Event ID), CF (Etalon), CF (Skill Previous p-Known), CF (Skill New p-Known), CF (Module), CF (Workspace Encounter), CF (Workspace Variant), CF (Workspace Progress Status), CF (Skill Opportunity), cf_school_id, cf_class_id, Selection <- ifelse(Action == "Done Button", "Done Button", NA), helpedTransaction, Gaming]
Index: []

[0 rows x 28 columns]
Updated LiveLab.csv with Gaming column saved as LiveLab_updated.csv


In [ ]:
import pandas as pd

# Step 1: Load the LiveLab.csv file
livelab_df = pd.read_csv('PreHelp_LiveLab.csv')

# Step 2: Load and parse the analysis.txt file
gaming_data = []
with open('Analysis-gaming-clip4-x019160_output.txt', 'r') as file:
    lines = file.readlines()
    for line in lines[1:]:  # Skip the header
        parts = line.strip().split('\t')  # Split by tab
        student_id = parts[0]  # Student ID
        time = parts[1]  # Time
        true_value = parts[3]  # The TRUE value (e.g., "Gaming" or "Not gaming")
        gaming_status = true_value == 'Gaming'  # True if 'Gaming', False otherwise
        gaming_data.append((student_id, time, gaming_status))

# Convert the parsed data into a DataFrame
gaming_df = pd.DataFrame(gaming_data, columns=['Anon Student Id', 'Time', 'IsGaming'])

# Convert the 'Time' column in both DataFrames to datetime for accurate comparison
# Ensure both are timezone-naive or timezone-aware (consistent format)
livelab_df['Time'] = pd.to_datetime(livelab_df['Time'], utc=True).dt.tz_localize(None)  # Remove timezone
gaming_df['Time'] = pd.to_datetime(gaming_df['Time'], utc=True).dt.tz_localize(None)  # Remove timezone

# Step 3: Merge the two DataFrames on 'Anon Student Id' and 'Time'
merged_df = pd.merge(livelab_df, gaming_df, on=['Anon Student Id', 'Time'], how='left')

# Step 4: Fill NaN values in 'IsGaming' with False (since NaN means no gaming activity was recorded)
merged_df['IsGaming'] = merged_df['IsGaming'].fillna(False)

# Step 5: Save the updated DataFrame to a new CSV file
merged_df.to_csv('LiveLab_with_Gaming.csv', index=False)

# Print values where Gaming is True
gaming_true_df = merged_df[merged_df["IsGaming"] == True]
print(gaming_true_df)